Please download "F-F_Research_Data_Factors_daily.CSV" from the course site and place it in the same directory as this notebook to run this code.

In [ ]:
import pandas as pd
import yfinance as yf
import numpy as np 
import statsmodels.api as sm 

def load_ff3():
    ff3 = pd.read_csv('F-F_Research_Data_Factors_daily.CSV')
    ff3['Unnamed: 0']=pd.DatetimeIndex(ff3['Unnamed: 0'].map(str))
    ff3 = ff3.set_index('Unnamed: 0')
    return ff3[['Mkt-RF','SMB','HML']] / 100.
ret = load_ff3()

arkk = yf.download('ARKK')['Adj Close']
arkk.index = arkk.index.date
ret['ARKK'] = arkk.pct_change()
ret = ret.dropna()


In [ ]:
(ret['ARKK']+1).cumprod().plot()

In [ ]:
ret['ARKK'].mean() / ret['ARKK'].std() * np.sqrt(252)

In [ ]:
X = ret[['Mkt-RF','SMB','HML']]
X = sm.add_constant(X)

mod = sm.OLS( ret['ARKK'], X)
res = mod.fit()
print(res.summary())

In [ ]:
# loadings
res.params

In [ ]:
# r-squared 
res.predict(X).var() / ret['ARKK'].var()

In [ ]:
# risk contribution 
X.multiply(res.params).std()[['Mkt-RF','SMB','HML']] / ret['ARKK'].std()

In [ ]:
# x-std shock
(X[['Mkt-RF','SMB','HML']].std()*5)*res.params[['Mkt-RF','SMB','HML']].abs()

In [ ]:
# tstats
res.tvalues

In [ ]:
# neutralized portfolio 
neut = ret['ARKK'] - X.multiply(res.params)[['Mkt-RF','SMB','HML']].sum(1)
neut

In [ ]:
neut.mean() / neut.std() * np.sqrt(252)

In [ ]:
# rolling betas for neutralization
def get_betas(ret):
    X = ret[['Mkt-RF','SMB','HML']]
    X = sm.add_constant(X)
    mod = sm.OLS( ret['ARKK'], X)
    res = mod.fit()
    return res.params 

betas = {}
for dt in ret.index:
    start_dt = dt - pd.tseries.offsets.Day()*252
    sub_ret = ret.loc[start_dt:dt]
    if len(sub_ret) > 60:
        betas[dt] = get_betas(sub_ret)
betas = pd.DataFrame(betas).T

In [ ]:
betas.plot()

In [ ]:
betas